<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Interacting with Azure SQL Servers - Part 2

## Azure Actions Covered

* Connecting to an Azure SQL Server with pyodbc
* Updating data in a SQL database
* Deleting data from a SQL database

In this lecture, we'll learn how to interact with Azure SQL Servers with Python.

To begin, we'll need to import our usual libraries as well as any useful environment variables (e.g. AZURE_SUBSCRIPTION_ID).

In [1]:
from azure.identity import AzureCliCredential
from azure.mgmt.sql import SqlManagementClient

import pyodbc

from settings import AZURE_SUBSCRIPTION_ID, DEFAULT_LOCATION, DEFAULT_RESOURCE_GROUP

We'll instantiate our credential as well as our `SqlManagementClient`.

In [2]:
credential = AzureCliCredential()
sql_client = SqlManagementClient(credential, AZURE_SUBSCRIPTION_ID)

Let's set up the connection string again for our SQL Server.

In [3]:
SERVER = 'bens-server.database.windows.net'
DATABASE = 'bens-db'
USERNAME = 'benadmin'
PASSWORD = 'testpassword123!'
# Has to be installed
DRIVER = '{ODBC Driver 18 for SQL Server}'

connection_string = f'DRIVER={DRIVER};PORT=1433;SERVER={SERVER};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD}'

Then we can set up our connection and cursor.

In [17]:
connection = pyodbc.connect(connection_string)
cursor = connection.cursor()

## Updating Data

Updating data in a SQL Server database works much the same way as inserting data. All that changes is the SQL statement itself. We still have to first execute and then commit it.

In [31]:
update_stmt = cursor.execute("""
update [SalesLT].[Address]
    set [SalesLT].[Address].[AddressLine1] = ?
    where [SalesLT].[Address].[AddressID] = ?
""",
'55 Easy Street', 11383)
connection.commit()

Let's check the `rowcount` attribute again to validate how many rows were updated.

In [32]:
update_stmt.rowcount

1

As we did in the section on inserting data, let's select the data we updated.

In [33]:
updated_results = cursor.execute(
"""
select *
from [SalesLT].[Address]
where [SalesLT].[Address].[AddressLine1] = ?
""", '55 Easy Street'
).fetchall()

In [34]:
updated_results

[(11383, '55 Easy Street', '', 'Austin', 'Tennessee', 'United States', '55667', '2670B2E6-DE5D-418D-8417-453F95547865', datetime.datetime(2023, 4, 26, 16, 35, 26, 670000))]

## Deleting Data

Deleting data via `pyodbc` follows the same pattern as inserting and updating. First execute, then commit.

In [35]:
delete_stmt = cursor.execute("""
delete
    from [SalesLT].[Address]
    where [SalesLT].[Address].[AddressID] = ?
""",
11383)
connection.commit()

Again, call the `rowcount` attribute to see how many rows were deleted.

In [36]:
delete_stmt.rowcount

1

Let's see if we can still retrieve the data that was deleted. `fetchall()` should return an empty list.

In [37]:
cursor.execute(
"""
select *
from [SalesLT].[Address]
where [SalesLT].[Address].[AddressLine1] = ?
""", '55 Easy Street'
).fetchall()

[]